In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import time
import cPickle
import os
from joblib import Parallel, delayed
import multiprocessing
# from optparse import OptionParser

# parser = OptionParser()
# parser.add_option("-d", "--dir", dest="data_dir", help="root folder with all XML files", metavar="DIR")
# parser.add_option("-r", "--root_dir", dest="root_path", help="root folder with all XML files",
#                   default='G:\My Drive\CRG_Dropbox\AljoComputer\Matlab\high_res_tracker\Data',
#                   metavar="DIR")

def getFrameData(root, iterKey):
    allFrameData = []
    for frameData in root.iter(iterKey):
        if not frameData.attrib:
            allFrameData.append(frameData.text)
        else:
            allFrameData.append(frameData.attrib)
    return allFrameData

def readRuleData(file_name):
    tree = ET.parse(file_name)
    root = tree.getroot()
    parent = {}
    for child in root:
        tempList = list(child)
    #     print child.tag
        if not tempList:
            if not child.attrib:
                parent[child.tag] = child.text
            else:
                parent[child.tag] = child.attrib
        else:
            parent[child.tag] = {}
            for grand_child in child:
    #             print '\t', grand_child.tag
                tempList = list(grand_child)
                if not tempList:
                    if not grand_child.attrib:
                        parent[child.tag][grand_child.tag] = grand_child.text
                    else:
                        parent[child.tag][grand_child.tag] = grand_child.attrib
                else:
                    parent[child.tag][grand_child.tag] = {}
                    for grand_grand_child in grand_child:
    #                     print '\t\t', grand_grand_child.tag
                        tempList = list(grand_grand_child)
                        if not tempList:
                            if not grand_grand_child.attrib:
                                parent[child.tag][grand_child.tag][grand_grand_child.tag] = grand_grand_child.text
                            else:
                                parent[child.tag][grand_child.tag][grand_grand_child.tag] = grand_grand_child.attrib
                        else:
                            parent[child.tag][grand_child.tag][grand_grand_child.tag] = {}
                            for grand_grand_grand_child in grand_grand_child:
    #                             print '\t\t\t', grand_grand_grand_child.tag
                                tempList = list(grand_grand_grand_child)
                                if not tempList:
                                    if not grand_grand_grand_child.attrib:
                                        parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.text
                                    else:
                                        parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.attrib
                                else:
                                    parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = {}
                                    for grand_grand_grand_grand_child in grand_grand_grand_child:
            #                             print '\t\t\t', grand_grand_grand_grand_child.tag
                                        tempList = list(grand_grand_grand_grand_child)
                                        if not tempList:
                                            if not grand_grand_grand_grand_child.attrib:
                                                parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.text
                                            else:
                                                parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.attrib
                                        else:
                                            print "There are more grand childrens than you think"
    parent['ruleData'] = []
    parent['ruleData'] = pd.DataFrame(getFrameData(root, 'ruleData'))
    
    return parent

def readKinData(file_name, inum):
    print 'Parsing File Number ', inum
    tree = ET.parse(file_name)
    root = tree.getroot()
    
    kinData = []
    kinetics = []
    stimulus = []

    larvaFrameData = pd.DataFrame(getFrameData(root, 'larvaFrameData'))
    skeleton = pd.DataFrame(getFrameData(root, 'skeleton'))
    kinetics = pd.concat([larvaFrameData, skeleton], axis=1)
    midpoint = pd.DataFrame(getFrameData(root, 'midpoint'), columns=['mid_x', 'mid_y'])
    centroid = pd.DataFrame(getFrameData(root, 'centroid'), columns=['centroid_x', 'centroid_y'])
    head = pd.DataFrame(getFrameData(root, 'head'), columns=['head_x', 'head_y'])
    tail = pd.DataFrame(getFrameData(root, 'tail'), columns=['tail_x', 'tail_y'])
    featureLocation = pd.DataFrame(getFrameData(root, 'featureLocation'))
    
    stimulusIntensity = pd.DataFrame(getFrameData(root, 'intensityPercentage'))
    temp = pd.DataFrame(getFrameData(root, 'ledArrayStimulus'))
    if not temp.empty:
        stimulus = pd.concat([temp, stimulusIntensity], axis=1)
        stimulus.rename(columns={0: 'intensity'}, inplace=True)
    else:
        stimulus = stimulusIntensity
    
    kinData = pd.concat([kinetics, midpoint, centroid, head, tail, stimulus], axis=1)
    kinData['num'] = inum
    kinData['timeIndex'] = kinData.index.values

    return kinData


In [2]:
if __name__ == '__main__':
#     (options, args) = parser.parse_args()
    num_cores = multiprocessing.cpu_count()
    root_path = '/Volumes/GoogleDrive/My Drive/CRG_Dropbox/AljoComputer/Matlab/high_res_tracker/Data'
    data_dir = '20180321_OK371_CsChrimson_Midpoint_DoseResponse_200micron'
    
    all_xml_files = []
    data_dir = os.path.join(root_path.strip(), data_dir.strip(), 'XML')
    for file in os.listdir(data_dir):
        if file.endswith(".xml"):
            all_xml_files.append(os.path.join(data_dir, file))

    print "Total XML Files Found - ", len(all_xml_files)

    ## Parse Files in Parallel
    tStart = time.time()
    kin_data = []
    kin_data = Parallel(n_jobs=num_cores)(delayed(readKinData)(xml_file, idx) for idx, xml_file in enumerate(all_xml_files))
    print 'Time taken to parse %d files in parallel is %0.3f sec'%(len(all_xml_files), time.time()-tStart)

    ## Parse Rule Data
    tStart = time.time()
    rule_data = []
    rule_data = readRuleData(all_xml_files[0])
    print 'Time taken to parse rule data is %0.3f sec' %(time.time()-tStart)

    kin_data_all = []
    kin_data_all = pd.concat(kin_data)
    kin_data_all.reset_index(inplace=True, drop=True)
    kin_data_all.loc[kin_data_all['behaviorMode']=='RUN', 'behaviorMode'] = 1
    kin_data_all.loc[kin_data_all['behaviorMode']=='TURN_LEFT', 'behaviorMode'] = 2
    kin_data_all.loc[kin_data_all['behaviorMode']=='TURN_RIGHT', 'behaviorMode'] = 3
    kin_data_all.loc[kin_data_all['behaviorMode']=='STOP', 'behaviorMode'] = 4
    kin_data_all.loc[kin_data_all['behaviorMode']=='CAST_LEFT', 'behaviorMode'] = 5
    kin_data_all.loc[kin_data_all['behaviorMode']=='CAST_RIGHT', 'behaviorMode'] = 6
    kin_data_all.loc[kin_data_all['behaviorMode']=='BACK_UP', 'behaviorMode'] = 7
    kin_data_all.loc[kin_data_all['behaviorMode'].isnull(), 'behaviorMode'] = 0
    cols = kin_data_all.columns.tolist()
    cols = cols[-2:]+ cols[1:-2]
    kin_data_all=kin_data_all[cols]

    ## Save Parsed Files
    print 'Saving as parsed data'
    tStart = time.time()
    kin_data_all.to_csv(open(os.path.join(data_dir, 'kinData_xml.dat'), 'wb'), sep='\t', index=False)
    cPickle.dump(rule_data, open(os.path.join(data_dir, 'ruleData_xml.pkl'), 'wb'), protocol=2)
    print 'Time taken to save parsed data is %0.3f sec' % (time.time() - tStart)

Total XML Files Found -  2
Parsing File Number  0
Parsing File Number  1
Time taken to parse 2 files in parallel is 33.364 sec
Time taken to parse rule data is 23.833 sec
Saving as parsed data
Time taken to save parsed data is 0.817 sec


,num,timeIndex,allocentricHeadAngle,allocentricHeadAngleSpeed,behaviorMode,bodyAngleSpeed,centroidSpeed,derivedMaxLength,headAngleSpeed,headSpeed,...,head_y,tail_x,tail_y,color,height,rotation,width,x,y,intensity
0,0,0,0.0,0.0,4,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,CRM,0.2,0.0,0.2,0.0,0.0,0.0
1,0,1,NaN,NaN,4,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,CRM,0.2,0.0,0.2,0.0,0.0,0.0
2,0,2,NaN,NaN,4,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,CRM,0.2,0.0,0.2,0.0,0.0,0.0
3,0,3,NaN,NaN,4,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,CRM,0.2,0.0,0.2,0.0,0.0,0.0
4,0,4,NaN,NaN,4,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,CRM,0.2,0.0,0.2,0.0,0.0,0.0
5,0,5,NaN,NaN,4,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,CRM,0.2,0.0,0.2,0.0,0.0,0.0
6,0,6,NaN,NaN,4,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,CRM,0.2,0.0,0.2,0.0,0.0,0.0
7,0,7,NaN,NaN,4,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,CRM,0.2,0.0,0.2,0.0,0.0,0.0
8,0,8,NaN,NaN,4,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,CRM,0.2,0.0,0.2,0.0,0.0,0.0
9,0,9,NaN,NaN,4,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,CRM,0.2,0.0,0.2,0.0,0.0,0.0
